# Conv3D Neural Network

In this part, I build a Conv3D neural network, then I train this model with 1320 data, including 1188 training data and 132 test data(choose training data/test data = 9/1), and I save the model to HDF5 file.

In [1]:
import tensorflow as tf

from keras import Sequential
from keras.layers import Conv3D, MaxPooling3D, Dense, Flatten, Dropout
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras.models import model_from_json
from sklearn.cross_validation import train_test_split

import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

d:\programs\anaconda\envs\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
d:\programs\anaconda\envs\python35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Number of frames of a video and number of classes
nframes = 30
nclasses = 20

In [11]:
# Clear session
K.clear_session()

In [12]:
# construct deep network
model = Sequential()
model.add(Conv3D(input_shape=(nframes, 128, 128, 3), 
                 filters=32, 
                 kernel_size=(7, 7, 7), 
                 padding='valid', 
                 strides=(2, 4, 4), 
                 data_format="channels_last",
                 activation="relu"))
model.add(MaxPooling3D(pool_size=(3, 3, 3),
                       strides=(1, 2, 2),
                       padding='valid',
                       data_format='channels_last'))
model.add(Conv3D(filters=64, 
                 kernel_size=(5, 5, 5), 
                 padding='valid', 
                 strides=(1, 2, 2),
                 data_format="channels_last",
                 activation="relu"))
model.add(MaxPooling3D(pool_size=(3, 3, 3),
                       strides=1,
                       padding='valid',
                       data_format='channels_last'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nclasses, activation='softmax'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 12, 31, 31, 32)    32960     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 15, 15, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 6, 6, 6, 64)       256064    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 4, 4, 4, 64)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 4, 64)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               524416    
__________

In [14]:
model.compile(loss="categorical_crossentropy",
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [7]:
def appendlabel(label, Yset):
    if int(label) == 20:
        Yset.append(11)
    elif int(label) == 30:
        Yset.append(12)
    elif int(label) == 40:
        Yset.append(13)
    elif int(label) == 50:
        Yset.append(14)
    elif int(label) == 60:
        Yset.append(15)
    elif int(label) == 70:
        Yset.append(16)
    elif int(label) == 80:
        Yset.append(17)
    elif int(label) == 90:
        Yset.append(18)
    elif int(label) == 100:
        Yset.append(19)
    else:
        Yset.append(int(label))

In [8]:
def sort(obj):
    aux = []
    for i in obj:
        name = i.split(".")[0]
        aux.append(int(name))
    aux.sort()
    for j in range(len(obj)):
        obj[j] = str(aux[j])+".jpg"

In [9]:
# Load Data
data_path = "./Frame_Data"
Xset = []
Yset = []

for i in os.listdir(data_path):
    label_path = os.path.join(data_path, i)
    for j in os.listdir(label_path):
        person_path = os.path.join(label_path, j)
        serial = []
        imgList = os.listdir(person_path)
        sort(imgList)
        for k in imgList:
            if k.endswith(".jpg"):
                img = Image.open(os.path.join(person_path, k))
                img = np.array(img)
                serial.append(img)
        Xset.append(serial)
        appendlabel(i, Yset)
        
# pre-processing data
Xset = np.array(Xset)
Xset = Xset.astype('float32')
Xset -= np.mean(Xset)
Xset /= np.max(Xset)

Yset = np.array(Yset)
Yset = to_categorical(Yset)

In [10]:
# Split training data and test data
Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xset, Yset, test_size=0.1)

In [15]:
# Train model
batch_size = 128
model.fit(Xtrain, Ytrain, 
          validation_data=(Xtest, Ytest),
          batch_size=batch_size,
          epochs=100,
          shuffle=True)

Train on 1188 samples, validate on 132 samples
Epoch 1/100
1188/1188 [==============================] - 30s 25ms/step - loss: 4.2333 - acc: 0.0446 - val_loss: 2.9960 - val_acc: 0.0530
Epoch 2/100
1188/1188 [==============================] - 7s 6ms/step - loss: 2.9969 - acc: 0.0387 - val_loss: 2.9975 - val_acc: 0.0455
Epoch 3/100
1188/1188 [==============================] - 7s 6ms/step - loss: 3.0161 - acc: 0.0539 - val_loss: 3.0006 - val_acc: 0.0152
Epoch 4/100
1188/1188 [==============================] - 7s 6ms/step - loss: 2.9974 - acc: 0.0690 - val_loss: 2.9936 - val_acc: 0.0455
Epoch 5/100
1188/1188 [==============================] - 7s 6ms/step - loss: 2.9641 - acc: 0.0749 - val_loss: 2.9219 - val_acc: 0.1136
Epoch 6/100
1188/1188 [==============================] - 7s 6ms/step - loss: 2.9197 - acc: 0.1019 - val_loss: 2.9682 - val_acc: 0.0606
Epoch 7/100
1188/1188 [==============================] - 7s 6ms/step - loss: 2.9114 - acc: 0.1002 - val_loss: 2.6685 - val_acc: 0.1288
Epoch 

Epoch 61/100
1188/1188 [==============================] - 7s 6ms/step - loss: 0.0908 - acc: 0.9781 - val_loss: 0.0019 - val_acc: 1.0000
Epoch 62/100
1188/1188 [==============================] - 7s 6ms/step - loss: 0.0687 - acc: 0.9731 - val_loss: 0.1341 - val_acc: 0.9773
Epoch 63/100
1188/1188 [==============================] - 7s 6ms/step - loss: 0.0874 - acc: 0.9773 - val_loss: 0.9084 - val_acc: 0.8712
Epoch 64/100
1188/1188 [==============================] - 7s 6ms/step - loss: 0.2769 - acc: 0.9461 - val_loss: 0.1781 - val_acc: 0.9545
Epoch 65/100
1188/1188 [==============================] - 7s 6ms/step - loss: 0.0918 - acc: 0.9781 - val_loss: 0.0369 - val_acc: 0.9773
Epoch 66/100
1188/1188 [==============================] - 7s 6ms/step - loss: 0.1456 - acc: 0.9663 - val_loss: 0.0074 - val_acc: 1.0000
Epoch 67/100
1188/1188 [==============================] - 7s 6ms/step - loss: 0.1762 - acc: 0.9579 - val_loss: 0.1583 - val_acc: 0.9773
Epoch 68/100
1188/1188 [========================

In [16]:
# Train next 100 epoch if first 100 epoch didn't converge well
# model.fit(Xtrain, Ytrain, 
#           validation_data=(Xtest, Ytest),
#           batch_size=batch_size,
#           epochs=200,
#           initial_epoch=100,
#           shuffle=True)

In [17]:
# Evaluate on test data
score = model.evaluate(Xtest, Ytest, batch_size=batch_size)
print("loss on test data: %.6f%%" % (score[0]*100))
print("accuracy on test data: %.2f%%" % (score[1]*100))

132/132 [==============================] - 1s 5ms/step
loss on test data: 1.811191%
accuracy on test data: 99.24%


In [18]:
# Save model weigth
# serialize model to JSON
model_json = model.to_json()
with open("./Weight/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./Weight/model.h5")